In [ ]:
import xarray as xr
import hvplot.xarray
import cartopy.crs as ccrs

In [ ]:
ds = xr.open_dataset('/global/cscratch1/sd/twhilton/WRFv4.1_Experiments/BTRAN_tests/BTRAN_test_001/WRFV4/run/output/metem_BTRAN000_d02_2005-06-01_01:30:00.nc')

In [ ]:
ds['XLAT'].attrs['long_name'] = 'latitude'
ds['XLONG'].attrs['long_name'] = 'longitude'
ds['XLAT'].attrs['units'] = 'deg N'
ds['XLONG'].attrs['units'] = 'deg E'

In [ ]:
ds['BTRAN'].sel(Time=0).hvplot.quadmesh(y='XLAT', x='XLONG', 
                                        coastline='50m',
                                        projection=ccrs.PlateCarree(),
                                       title=ds['BTRAN'].attrs['description'])

In [ ]:
dsm = xr.open_mfdataset(('/global/cscratch1/sd/twhilton/WRFv4.1_Experiments/BTRAN_tests/BTRAN_test_001/WRFV4/run/output/metem_BTRAN000_d02_2005-06-01_01:00:00.nc',
                    '/global/cscratch1/sd/twhilton/WRFv4.1_Experiments/BTRAN_tests/BTRAN_test_001/WRFV4/run/output/metem_BTRAN000_d02_2005-06-01_01:30:00.nc'))

In [ ]:
dsm['d'] = ds['BTRAN'].sel(Time=0) - ds['BTRAN'].sel(Time=1)
dsm['d'].hvplot.quadmesh(title='01:00 - 01:30')